In [4]:
import pandas as pd
pd.set_option('display.max_rows', 100)
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from datetime import datetime
import numpy as np

In [3]:
news = pd.read_csv('bitcoin_news.csv')
news.head()

,Date,Summary,Tags,Title
0,2018-09-04T08:00:04-04:00,The Tuesday edition of our daily roundup is ja...,"ATM,Blockchain,Brave,Deutsche Börse,Ethereum,G...","The Daily: Wirex Adds Ethereum, Deutsche Börse..."
1,2018-08-02T17:15:26-04:00,Japan’s SBI Group is reportedly planning to cr...,"ATM,BCH,Bitcoin,bitcoin cash,BTC,BTM,coin asse...","SBI Plans Derivatives Platform, Huobi Eyes 30%..."
2,2018-09-05T01:15:31-04:00,Non-custodial crypto trading platform Shapeshi...,"AML,anti-money laundering,Bitcoin,BTC,crypto,C...",Shapeshift Moves to Membership Model Requiring...
3,2018-08-04T04:55:54-04:00,"Intercontinental Exchange (ICE), owner of argu...","401k,Bitcoin,Boston Consulting Group,Canada,CF...",NYSE Owner: Bitcoin Should Be in Retirement Fu...
4,2018-08-06T03:45:21-04:00,Thailand’s central bank has announced the rule...,"bank of thailand,banks,Bitcoin,Bot,BTC,commerc...",Bank of Thailand Green-Lights Financial Compan...


In [7]:
news['Datetime'] = list(map(lambda x: datetime.strptime(x[0:10],'%Y-%m-%d'), news['Date']))
news['Year'] = list(map(lambda x: str(datetime.strptime(x[0:10],'%Y-%m-%d').year).zfill(4), news['Date']))
news['Month'] = list(map(lambda x: str(datetime.strptime(x[0:10],'%Y-%m-%d').month).zfill(2), news['Date']))
news['Day'] = list(map(lambda x: str(datetime.strptime(x[0:10],'%Y-%m-%d').day).zfill(2), news['Date']))
news['Numerical_date'] = news['Year'] + news['Month'] + news['Day']
news['Numerical_date'] = [int(i) for i in news['Numerical_date']]
news['Textchunk'] = news['Summary'] + str(' ') + news['Title']
news.head()

,Date,Summary,Tags,Title,Datetime,Year,Month,Day,Numerical_date,Textchunk
0,2018-09-04T08:00:04-04:00,The Tuesday edition of our daily roundup is ja...,"ATM,Blockchain,Brave,Deutsche Börse,Ethereum,G...","The Daily: Wirex Adds Ethereum, Deutsche Börse...",2018-09-04,2018,09,04,20180904,The Tuesday edition of our daily roundup is ja...
1,2018-08-02T17:15:26-04:00,Japan’s SBI Group is reportedly planning to cr...,"ATM,BCH,Bitcoin,bitcoin cash,BTC,BTM,coin asse...","SBI Plans Derivatives Platform, Huobi Eyes 30%...",2018-08-02,2018,08,02,20180802,Japan’s SBI Group is reportedly planning to cr...
2,2018-09-05T01:15:31-04:00,Non-custodial crypto trading platform Shapeshi...,"AML,anti-money laundering,Bitcoin,BTC,crypto,C...",Shapeshift Moves to Membership Model Requiring...,2018-09-05,2018,09,05,20180905,Non-custodial crypto trading platform Shapeshi...
3,2018-08-04T04:55:54-04:00,"Intercontinental Exchange (ICE), owner of argu...","401k,Bitcoin,Boston Consulting Group,Canada,CF...",NYSE Owner: Bitcoin Should Be in Retirement Fu...,2018-08-04,2018,08,04,20180804,"Intercontinental Exchange (ICE), owner of argu..."
4,2018-08-06T03:45:21-04:00,Thailand’s central bank has announced the rule...,"bank of thailand,banks,Bitcoin,Bot,BTC,commerc...",Bank of Thailand Green-Lights Financial Compan...,2018-08-06,2018,08,06,20180806,Thailand’s central bank has announced the rule...


# word2vec

In [9]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

class MySentences(object): 
    def __iter__(self):
        for i in range(news.shape[0]):
            yield simple_preprocess(news.iloc[i,9])

C:\Users\USER\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
assert gensim.models.doc2vec.FAST_VERSION > -1

In [11]:
%%time
import multiprocessing
import os
cores = multiprocessing.cpu_count()

if not os.path.exists('models/word2vec.model'):
    print("start traing word2vec model...")
    sentences = MySentences()
    word2vec_model = gensim.models.Word2Vec(sentences, min_count=20, size=200, workers=cores)
    if not os.path.exists('models'):
        os.makedirs('models')
        word2vec_model.save('models/word2vec.model')
    else:
        word2vec_model.save('models/word2vec.model')
else:
    word2vec_model = Word2Vec.load('models/word2vec.model')

start traing word2vec model...
Wall time: 6.04 s


In [12]:
word2vec_model.wv.word_vec('japan')

array([ 0.23099245, -0.40995488, -0.07226684, -0.09342019, -0.2073103 ,
       -0.28695744, -0.37862435,  0.10322383,  0.2577616 ,  0.00439262,
       -0.07103091,  0.25559068,  0.27814412, -0.44493124,  0.23690538,
        0.35899192,  0.04432926, -0.3824167 , -0.09529694, -0.21771577,
        0.3447758 , -0.3188712 ,  0.04937246,  0.13526675, -0.36076596,
        0.1755127 ,  0.02825051, -0.0269706 , -0.00277254, -0.32282144,
        0.02933267,  0.31863832, -0.05808569,  0.20616844, -0.09393807,
        0.20806278,  0.3890938 , -0.16519515,  0.06242054, -0.49173915,
        0.15142614, -0.20087399,  0.19304298,  0.19440058,  0.29722986,
       -0.05419963,  0.23453556, -0.30852005,  0.11068523,  0.30393246,
       -0.19527276, -0.36275053,  0.1592779 , -0.10435552, -0.04008956,
        0.31441516,  0.26298535, -0.2601682 , -0.09579869,  0.07881349,
        0.14664854, -0.1829456 , -0.3219531 ,  0.33483863,  0.10853055,
       -0.5998203 , -0.05526113, -0.10232022, -0.453147  ,  0.42

In [13]:
word2vec_model.wv.most_similar(positive=['japan'], negative=['regulation'], topn=100)

C:\Users\USER\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('trading', 0.4962330162525177),
 ('exchange', 0.48667895793914795),
 ('launched', 0.453437864780426),
 ('news', 0.4526098072528839),
 ('recently', 0.44129276275634766),
 ('recent', 0.4158521890640259),
 ('com', 0.4090777635574341),
 ('launch', 0.3981197476387024),
 ('commission', 0.3778550326824188),
 ('based', 0.3678431510925293),
 ('largest', 0.35360029339790344),
 ('million', 0.32774606347084045),
 ('week', 0.32407522201538086),
 ('volume', 0.3150397837162018),
 ('after', 0.3148190379142761),
 ('founder', 0.3030170798301697),
 ('futures', 0.30291688442230225),
 ('south', 0.3009553551673889),
 ('published', 0.2993319630622864),
 ('launches', 0.2982444763183594),
 ('announced', 0.2981336712837219),
 ('reported', 0.28790533542633057),
 ('hong', 0.28764626383781433),
 ('ceo', 0.2863397002220154),
 ('has', 0.2849130630493164),
 ('last', 0.2839438319206238),
 ('firm', 0.28222978115081787),
 ('securities', 0.2799607217311859),
 ('sec', 0.27879172563552856),
 ('company', 0.271463274955749

# doc2vec

In [17]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

class MyDocs(object):
    def __iter__(self):
        for i in range(news.shape[0]):
            yield TaggedDocument(words=simple_preprocess(news.iloc[i,9]), tags=['%d' % news.iloc[i,8]])

In [18]:
%%time

if not os.path.exists('models/doc2vec.model'):
    print("start traing doc2vec model...")
    documents = MyDocs()
    doc2vec_model = Doc2Vec(dm=1, dbow_words=1, vector_size=200, window=8, min_count=20, workers=cores)
    doc2vec_model.build_vocab(documents)
    doc2vec_model.train(documents, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
    if not os.path.exists('models'):
        os.makedirs('models')
        doc2vec_model.save('models/doc2vec.model')
    else:
        doc2vec_model.save('models/doc2vec.model')
else:
    doc2vec_model = Doc2Vec.load('models/doc2vec.model')

start traing doc2vec model...
Wall time: 10.4 s


In [23]:
doc2vec_model.docvecs.most_similar('20180118', topn = 100)

C:\Users\USER\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('20170927', 0.9927163124084473),
 ('20180315', 0.9926577806472778),
 ('20170512', 0.9926438331604004),
 ('20160519', 0.9924439191818237),
 ('20180217', 0.9924414157867432),
 ('20150822', 0.9910603761672974),
 ('20160926', 0.9904611110687256),
 ('20180507', 0.9899706840515137),
 ('20170704', 0.9898616075515747),
 ('20180727', 0.9896129369735718),
 ('20180115', 0.9895788431167603),
 ('20161113', 0.9895451068878174),
 ('20160529', 0.9893954396247864),
 ('20160723', 0.9893855452537537),
 ('20170121', 0.9893706440925598),
 ('20170830', 0.9892613887786865),
 ('20170908', 0.989223837852478),
 ('20161105', 0.9891835451126099),
 ('20161209', 0.9886587858200073),
 ('20160512', 0.9883790016174316),
 ('20180604', 0.988359808921814),
 ('20170706', 0.9882188439369202),
 ('20170925', 0.9880560636520386),
 ('20170711', 0.9879214763641357),
 ('20170126', 0.9878672957420349),
 ('20170505', 0.9877114295959473),
 ('20180607', 0.9875342845916748),
 ('20160603', 0.9874308109283447),
 ('20180328', 0.987167